# Analysis

In [151]:
%reload_ext autoreload
%autoreload 2

In [417]:
#export
from exp.nb_00 import *
from exp.nb_01 import *
from exp.nb_02 import *
import os
import shutil
import pickle
import re
import subprocess
import nbformat
from nbconvert.preprocessors import ExecutePreprocessor
import numpy as np
from climatools.lblnew.export import vector_to_F77

In [162]:
param = LBLnewBestfitSWParam(band=9, vmin=8200, vmax=14290, nv=10000, dv=0.001,
                     molecule='h2o', conc='atmpro',
                     ref_pts=[(300, 250), (300, 250)], ng_refs=[4, 6], ng_adju=[-3, 0],
                     wgt=[(.95, .9, .5, .5), (.5, .5, .5, .7, .85, .95)],
                     cosz=1., rsfc=0,
                     klin=3e-25,
                     option_k_lookup=0)

In [114]:
runner = LBLnewBestfitSWRun('test1/test1', param)

In [115]:
list(runner.path.iterdir())

[PosixPath('test1/test1/analysis.ipynb'),
 PosixPath('test1/test1/lblnew-bestfit-sw.f'),
 PosixPath('test1/test1/fort.400'),
 PosixPath('test1/test1/fort.401'),
 PosixPath('test1/test1/fort.402'),
 PosixPath('test1/test1/lblnew-bestfit-sw.sub'),
 PosixPath('test1/test1/.ipynb_checkpoints'),
 PosixPath('test1/test1/fort.403'),
 PosixPath('test1/test1/fort.19'),
 PosixPath('test1/test1/fort.20'),
 PosixPath('test1/test1/lblnew-bestfit-sw.exe'),
 PosixPath('test1/test1/lblcom.f'),
 PosixPath('test1/test1/fort.10'),
 PosixPath('test1/test1/fort.9'),
 PosixPath('test1/test1/err_432747'),
 PosixPath('test1/test1/out_432747')]

## Set up analysis directory

In [418]:
PATH_ANALYSIS

PosixPath('/chia_cluster/home/jackyu/radiation/analysis_-_new_kdist_param_sw')

In [419]:
# We are going to test directly at a location inside the 'analysis_-_new_kdist_param_sw' repository, 
# because we will try to git-add and git-commit the executed notebook.
ANA_PLACE = PATH_ANALYSIS

In [347]:
# This is the directory to save the analysis notebook in.
PATH = ANA_PLACE/'analysis_test'

In [348]:
PATH.mkdir(exist_ok=True, parents=True)

## Load the analysis notebook template

In [349]:
! pwd

/chia_cluster/home/jackyu/radiation/analysis_-_new_kdist_param_sw


In [350]:
os.chdir('/chia_cluster/home/jackyu/climatools/climatools/notebooks/sw_dev/')

In [351]:
# For now, we assume that the analysis notebook template resides in the current directory
# in which we're developing things: /chia_cluster/home/jackyu/climatools/climatools/notebooks/sw_dev/.
with open('analysis_-_lblnew-bestfit-sw.ipynb') as f:
    analysis_nb = nbformat.read(f, as_version=nbformat.NO_CONVERT)

In [352]:
str(vars(runner.param))

"{'dv': 0.001, 'nv': 10000, 'commitnumber': None, 'band': 9, 'molecule': 'h2o', 'atmpro': None, 'tsfc': None, 'vmin': 8200, 'vmax': 14290, 'conc': 'atmpro', 'ref_pts': [(300, 250), (300, 250)], 'ng_refs': [4, 6], 'ng_adju': [-3, 0], 'wgt': [(0.95, 0.9, 0.5, 0.5), (0.5, 0.5, 0.5, 0.7, 0.85, 0.95)], 'cosz': 1.0, 'rsfc': 0, 'klin': 3e-25, 'option_k_lookup': 0}"

In [353]:
analysis_nb['cells'][2]['source'] = f'''PARAM = LBLnewBestfitSWParam(**{vars(runner.param)})'''
analysis_nb['cells'][2]

{'cell_type': 'code',
 'execution_count': 3,
 'metadata': {},
 'outputs': [],
 'source': "PARAM = LBLnewBestfitSWParam(**{'dv': 0.001, 'nv': 10000, 'commitnumber': None, 'band': 9, 'molecule': 'h2o', 'atmpro': None, 'tsfc': None, 'vmin': 8200, 'vmax': 14290, 'conc': 'atmpro', 'ref_pts': [(300, 250), (300, 250)], 'ng_refs': [4, 6], 'ng_adju': [-3, 0], 'wgt': [(0.95, 0.9, 0.5, 0.5), (0.5, 0.5, 0.5, 0.7, 0.85, 0.95)], 'cosz': 1.0, 'rsfc': 0, 'klin': 3e-25, 'option_k_lookup': 0})"}

In [354]:
analysis_nb['cells'][3]['source'] = f'''PATH = Path("{runner.path}")'''
analysis_nb['cells'][3]

{'cell_type': 'code',
 'execution_count': 4,
 'metadata': {'collapsed': True},
 'outputs': [],
 'source': 'PATH = Path("test1/test1")'}

In [355]:
Path(runner.path)

PosixPath('test1/test1')

## Run the analysis notebook  
Useful: https://nbconvert.readthedocs.io/en/latest/execute_api.html

In [357]:
PATH, list(PATH.iterdir())

(PosixPath('/chia_cluster/home/jackyu/radiation/analysis_-_new_kdist_param_sw/analysis_test'),
 [])

In [358]:
ep = ExecutePreprocessor(timeout=600)
ep.preprocess(analysis_nb, {});

In [359]:
with open(PATH/'analysis_-_lblnew-bestfit-sw.ipynb', mode='w', encoding='utf-8') as f:
    nbformat.write(analysis_nb, f)

In [360]:
PATH

PosixPath('/chia_cluster/home/jackyu/radiation/analysis_-_new_kdist_param_sw/analysis_test')

## Git add the executed notebook

In [361]:
PATH, list(PATH.iterdir())

(PosixPath('/chia_cluster/home/jackyu/radiation/analysis_-_new_kdist_param_sw/analysis_test'),
 [PosixPath('/chia_cluster/home/jackyu/radiation/analysis_-_new_kdist_param_sw/analysis_test/analysis_-_lblnew-bestfit-sw.ipynb')])

In order to add the executed notebook to the 'analysis_-_lblnew-bestfit-sw' directory, we need to be under that directory at least. 

In [363]:
cwd = os.getcwd()
cwd

'/chia_cluster/home/jackyu/climatools/climatools/notebooks/sw_dev'

In [364]:
os.chdir(PATH)

In [365]:
! ls

analysis_-_lblnew-bestfit-sw.ipynb


In [370]:
proc = subprocess.Popen(['git', 'add', 'analysis_-_lblnew-bestfit-sw.ipynb'], 
                        stdout=subprocess.PIPE, stderr=subprocess.PIPE)

In [371]:
out, err = proc.communicate()

In [372]:
print(out.decode(), err.decode())

In [373]:
! git status

On branch master
Your branch is up to date with 'origin/master'.

Changes to be committed:
  (use "git reset HEAD <file>..." to unstage)

	new file:   analysis_-_lblnew-bestfit-sw.ipynb

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   ../atmpro_None/band_9/commitnumber_None/conc_atmpro/cosz_1.0/dv_0.001/klin_3e-25/molecule_h2o/ng_adju_[-3, 0]/ng_refs_[4, 6]/nv_10000/option_k_lookup_0/ref_pts_[(300, 250), (300, 250)]/rsfc_0/tsfc_None/vmax_14290/vmin_8200/wgt_[(0.95, 0.9, 0.5, 0.5), (0.5, 0.5, 0.5, 0.7, 0.85, 0.95)]/analysis_-_lblnew-bestfit-sw.ipynb
	modified:   ../band09_-_h2o_atmpro_-_mls_-_cosz_1.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	../.ipynb_checkpoints/



## Git-commit the analysis notebook 
The commit message will contain the parameters for this particular case.

In [374]:
runner.param

<class 'exp.nb_00.LBLnewBestfitSWParam'>
{'dv': 0.001, 'nv': 10000, 'commitnumber': None, 'band': 9, 'molecule': 'h2o', 'atmpro': None, 'tsfc': None, 'vmin': 8200, 'vmax': 14290, 'conc': 'atmpro', 'ref_pts': [(300, 250), (300, 250)], 'ng_refs': [4, 6], 'ng_adju': [-3, 0], 'wgt': [(0.95, 0.9, 0.5, 0.5), (0.5, 0.5, 0.5, 0.7, 0.85, 0.95)], 'cosz': 1.0, 'rsfc': 0, 'klin': 3e-25, 'option_k_lookup': 0}

In [380]:
title = f"band{runner.param.band:02d} {runner.param.molecule} {runner.param.atmpro} cosz={runner.param.cosz} nf_refs={runner.param.ng_refs}"
title

'band09 h2o None cosz=1.0 nf_refs=[4, 6]'

In [388]:
body = '\n'.join(sorted(f"{n} {v}" for n, v in vars(runner.param).items()))
print(body)

atmpro None
band 9
commitnumber None
conc atmpro
cosz 1.0
dv 0.001
klin 3e-25
molecule h2o
ng_adju [-3, 0]
ng_refs [4, 6]
nv 10000
option_k_lookup 0
ref_pts [(300, 250), (300, 250)]
rsfc 0
tsfc None
vmax 14290
vmin 8200
wgt [(0.95, 0.9, 0.5, 0.5), (0.5, 0.5, 0.5, 0.7, 0.85, 0.95)]


In [391]:
proc = subprocess.Popen(f'''git commit -m "{title}" -m "{body}"''', shell=True,
                        stdout=subprocess.PIPE, stderr=subprocess.PIPE)

In [392]:
out, err = proc.communicate()

In [393]:
! git status

On branch master
Your branch is ahead of 'origin/master' by 1 commit.
  (use "git push" to publish your local commits)

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   ../atmpro_None/band_9/commitnumber_None/conc_atmpro/cosz_1.0/dv_0.001/klin_3e-25/molecule_h2o/ng_adju_[-3, 0]/ng_refs_[4, 6]/nv_10000/option_k_lookup_0/ref_pts_[(300, 250), (300, 250)]/rsfc_0/tsfc_None/vmax_14290/vmin_8200/wgt_[(0.95, 0.9, 0.5, 0.5), (0.5, 0.5, 0.5, 0.7, 0.85, 0.95)]/analysis_-_lblnew-bestfit-sw.ipynb
	modified:   ../band09_-_h2o_atmpro_-_mls_-_cosz_1.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	../.ipynb_checkpoints/

no changes added to commit (use "git add" and/or "git commit -a")


In [394]:
! git log -1

commit 6fb47c1c797db1d3cc35dd5a3af515f290e4ef79 (HEAD -> master)
Author: Jack Yu <llacque@gmail.com>
Date:   Wed May 29 10:39:14 2019 +0800

    band09 h2o None cosz=1.0 nf_refs=[4, 6]
    
    atmpro None
    band 9
    commitnumber None
    conc atmpro
    cosz 1.0
    dv 0.001
    klin 3e-25
    molecule h2o
    ng_adju [-3, 0]
    ng_refs [4, 6]
    nv 10000
    option_k_lookup 0
    ref_pts [(300, 250), (300, 250)]
    rsfc 0
    tsfc None
    vmax 14290
    vmin 8200
    wgt [(0.95, 0.9, 0.5, 0.5), (0.5, 0.5, 0.5, 0.7, 0.85, 0.95)]


## Putting things together -- LBLnewBestfitSWAnalysis class

In [395]:
#export
class LBLnewBestfitSWAnalysis(object):
    def __init__(self, path, runner):
        self.path = Path(path)
        self.runner = runner
        self.path.mkdir(exist_ok=True, parents=True)
        
    def input_params(self):
        with open('analysis_-_lblnew-bestfit-sw.ipynb') as f:  # Need to set to stored absolute path
            nb = nbformat.read(f, as_version=nbformat.NO_CONVERT)
        nb['cells'][2]['source'] = f'''PARAM = LBLnewBestfitSWParam(**{vars(self.runner.param)})'''
        nb['cells'][3]['source'] = f'''PATH = Path("{self.runner.path}")'''
        return nb
        
    def run(self):
        nb = self.input_params()
        ep = ExecutePreprocessor(timeout=600)
        ep.preprocess(nb, {})
        with open(self.path/'analysis_-_lblnew-bestfit-sw.ipynb', mode='w', encoding='utf-8') as f:
            nbformat.write(nb, f)
            
    def gitcommit(self):
        cwd = os.getcwd()
        os.chdir(self.path)
        try:
            proc = subprocess.Popen(['git', 'add', 'analysis_-_lblnew-bestfit-sw.ipynb'], 
                                    stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            out, err = proc.communicate()
            print('Git-add', out.decode(), err.decode())
            title = f"band{self.runner.param.band:02d} {self.runner.param.molecule} {self.runner.param.atmpro} cosz={self.runner.param.cosz} nf_refs={self.runner.param.ng_refs}"
            body = '\n'.join(sorted(f"{n} {v}" for n, v in vars(self.runner.param).items()))
            proc = subprocess.Popen(f'''git commit -m "{title}" -m "{body}"''', shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            return proc
        finally: os.chdir(cwd)


In [397]:
os.chdir(cwd)

In [398]:
os.getcwd()

'/chia_cluster/home/jackyu/climatools/climatools/notebooks/sw_dev'

In [399]:
PATH

PosixPath('/chia_cluster/home/jackyu/radiation/analysis_-_new_kdist_param_sw/analysis_test')

In [400]:
analysis = LBLnewBestfitSWAnalysis(path=PATH, runner=runner)

In [401]:
analysis.run()

In [403]:
os.getcwd()

'/chia_cluster/home/jackyu/climatools/climatools/notebooks/sw_dev'

In [404]:
proc = analysis.gitcommit()

Git-add  


In [405]:
out, err = proc.communicate()

In [406]:
print('Git-commit', out.decode(), err.decode())

Git-commit [master 154c339] band09 h2o None cosz=1.0 nf_refs=[4, 6]
 1 file changed, 1821 insertions(+), 1821 deletions(-)
 rewrite analysis_test/analysis_-_lblnew-bestfit-sw.ipynb (74%)
 


In [409]:
os.getcwd()

'/chia_cluster/home/jackyu/climatools/climatools/notebooks/sw_dev'

In [410]:
cwd = os.getcwd()

In [412]:
os.chdir(PATH)

In [413]:
! ls

analysis_-_lblnew-bestfit-sw.ipynb


In [415]:
! git log -1

commit 154c339180a6afcc53a03ec7a163db6b62d11cce (HEAD -> master)
Author: Jack Yu <llacque@gmail.com>
Date:   Wed May 29 10:57:38 2019 +0800

    band09 h2o None cosz=1.0 nf_refs=[4, 6]
    
    atmpro None
    band 9
    commitnumber None
    conc atmpro
    cosz 1.0
    dv 0.001
    klin 3e-25
    molecule h2o
    ng_adju [-3, 0]
    ng_refs [4, 6]
    nv 10000
    option_k_lookup 0
    ref_pts [(300, 250), (300, 250)]
    rsfc 0
    tsfc None
    vmax 14290
    vmin 8200
    wgt [(0.95, 0.9, 0.5, 0.5), (0.5, 0.5, 0.5, 0.7, 0.85, 0.95)]


In [421]:
os.chdir(cwd)

# fin

In [425]:
! python notebook2script.py 03_analysis.ipynb

Converted 03_analysis.ipynb to exp/nb_03.py
